In [29]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [3]:
df_jan.columns.shape

(19,)

In [4]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan['duration'].dt.total_seconds() / 60
df_jan['duration'].std()

42.594351241920904

In [5]:
df_jan_outlier = df_jan[(df_jan['duration']) >=1 & (df_jan['duration']<=60)]
shape_new = df_jan_outlier.shape[0]
shape_before = df_jan.shape[0]

int((shape_new/shape_before) * 100)

98

In [6]:
df_copy = df_jan_outlier[['PULocationID', 'DOLocationID']]
df_copy['PULocationID'] = df_copy['PULocationID'].astype(str)
df_copy['DOLocationID'] = df_copy['DOLocationID'].astype(str)

dicts = df_copy.to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(dicts)

print(X.shape[1])

/tmp/ipykernel_18679/2680606014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['PULocationID'] = df_copy['PULocationID'].astype(str)
/tmp/ipykernel_18679/2680606014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['DOLocationID'] = df_copy['DOLocationID'].astype(str)


517


In [ ]:
y_train = df_jan_outlier['duration'].values

lr = LinearRegression()
lr.fit(X, y_train)
y_pred = lr.predict(X)

print(f'Train RMSE: {root_mean_squared_error(y_train, y_pred)}')